In [1]:
# 20180724 Customers Segmentation Engine / Arnaud ROUSSEAU

# This engine uses a .xls file with customers purchases descriptions to predict their 
# profiles just after the first shopping session.

# It uses a clean .csv file for the training.

#-----------------------------------------------------------------------------------------------------------------

# Libraries Import

import numpy as np
import pandas as pd

from sklearn import model_selection, preprocessing
from sklearn.ensemble import RandomForestClassifier

import datetime as dt

In [2]:
#-----------------------------------------------------------------------------------------------------------------

# Databases loading 

# Customer database for training 
data_client= pd.read_csv(
    'DATA_clean/data_Client.csv', 
    #sep='\t', 
    index_col=0,
    encoding='utf-8', 
    low_memory = False)


# Profiles to determine
data_client_pred = pd.read_excel('DATA_test/RFM_profiles.xlsx', sep=';')

# Test missing data
#data_client_pred = data_client_pred.dropna(axis = 0 , subset=['InvoiceNo']) 
#data_client_pred = data_client_pred.dropna(axis = 0 , subset=['StockCode']) 
#data_client_pred = data_client_pred.dropna(axis = 0 , subset=['Description']) 
data_client_pred = data_client_pred.dropna(axis = 0 , subset=['Quantity']) 
#data_client_pred = data_client_pred.dropna(axis = 0 , subset=['InvoiceDate']) 
data_client_pred = data_client_pred.dropna(axis = 0 , subset=['UnitPrice']) 
data_client_pred = data_client_pred.dropna(axis = 0 , subset=['CustomerID'])
#data_client_pred = data_client_pred.dropna(axis = 0 , subset=['Country'])

if data_client_pred.empty:
    print("Some data in Excel file is missing !")

In [3]:
#-----------------------------------------------------------------------------------------------------------------

# New Columns: Total Price, Quantity clone
data_client['TotalPrice'] = data_client['Quantity'] * data_client['UnitPrice']
data_client_pred['TotalPrice'] = data_client_pred['Quantity'] * data_client_pred['UnitPrice']

#-----------------------------------------------------------------------------------------------------------------

In [4]:
#-----------------------------------------------------------------------------------------------------------------
# Customers Profiling with RFM Score for TRAINING

NOW = dt.datetime(2011,12,10)
data_client['InvoiceDate'] = pd.to_datetime(data_client['InvoiceDate'])

rfmTable = data_client.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 
                                          'InvoiceNo': lambda x: len(x.unique()), 
                                          'TotalPrice': lambda x: x.sum(), 
                                           'Quantity': lambda x: x.sum(),
                                           })

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)


rfmTable.rename(columns={'InvoiceDate': 'Recency', 
                         'InvoiceNo': 'Frequency', 
                         'TotalPrice': 'Monetary_value',
                         'Quantity': 'Total_Quant',
                        }, 
                         inplace=True)

#-----------------------------------------------------------------------------------------------------------------

In [5]:
# Parameters calculation of Customers for PREDICTION
rfmTable_pred = data_client_pred.groupby('CustomerID').agg({'TotalPrice': lambda x: x.sum(),
                                                            'Quantity': lambda x: x.sum()
                                                           })

rfmTable_pred.rename(columns={'TotalPrice': 'Monetary_value',
                              'Quantity': 'Total_Quant'
                             }, 
                         inplace=True)

#-----------------------------------------------------------------------------------------------------------------

In [6]:
rfmTable_pred

,Monetary_value,Total_Quant
CustomerID,,
1,15.47,5
2,955.00,200
3,3815.00,900
4,5975.00,1500
5,13800.00,2000
6,24800.00,4000
10,940.00,1
11,4225.00,6500
12,7980.00,400


In [7]:
# Recency score creation
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

#-----------------------------------------------------------------------------------------------------------------
    
# Frequency and Monetary Score creation   
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

#-----------------------------------------------------------------------------------------------------------------

In [8]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

rfmTable['r_quartile'] = rfmTable['Recency'].apply(RScore, args=('Recency',quantiles))
rfmTable['f_quartile'] = rfmTable['Frequency'].apply(FMScore, args=('Frequency',quantiles))
rfmTable['m_quartile'] = rfmTable['Monetary_value'].apply(FMScore, args=('Monetary_value',quantiles))

rfmTable['RFMScore'] = rfmTable.r_quartile.map(str) \
                            + rfmTable.f_quartile.map(str) \
                            + rfmTable.m_quartile.map(str)

rfmTable = rfmTable.filter(items=(['Recency',
                                   'Frequency',
                                   'Monetary_value',
                                   'Total_Quant',
                                   'RFMScore'
                                  ]))

rfm_unique = rfmTable['RFMScore'].unique()

rfmTable.head()

,Recency,Frequency,Monetary_value,Total_Quant,RFMScore
CustomerID,,,,,
12346.0,325,1,77183.60,74215,441
12347.0,2,7,4310.00,2458,111
12348.0,75,4,1797.24,2341,321
12349.0,18,1,1757.55,631,241
12350.0,310,1,334.40,197,443


In [9]:
# List of the 10 most popular profiles
RFM_list_OK = ['111', '444', '443', '344', '222', '211', '122', '322', '244', '343']
rfmTable = rfmTable[rfmTable['RFMScore'].isin(RFM_list_OK)]

In [10]:
#-----------------------------------------------------------------------------------------------------------------

# Score RFM / Value association
def ScoreConv(x):
    if x == '111':
        return int(1)
    elif x == '444':
        return int(2)
    elif x == '443': 
        return int(3)
    elif x == '344': 
        return int(4)
    elif x == '222': 
        return int(5)
    elif x == '211': 
        return int(6)
    elif x == '122': 
        return int(7)
    elif x == '322': 
        return int(8)
    elif x == '244': 
        return int(9)
    elif x == '343': 
        return int(10)

In [11]:
#-----------------------------------------------------------------------------------------------------------------

rfmTable['RFM_Cat'] = rfmTable['RFMScore'].apply(lambda x: ScoreConv(x))

In [12]:
rfmTable['RFM_Cat'] = rfmTable['RFM_Cat'].astype(int)

In [13]:
rfmTable.head()

,Recency,Frequency,Monetary_value,Total_Quant,RFMScore,RFM_Cat
CustomerID,,,,,,
12347.0,2,7,4310.00,2458,111,1
12350.0,310,1,334.40,197,443,3
12352.0,36,7,1665.74,533,211,6
12353.0,204,1,89.00,20,444,2
12355.0,214,1,459.40,240,443,3


In [14]:
rfmTable_pred

,Monetary_value,Total_Quant
CustomerID,,
1,15.47,5
2,955.00,200
3,3815.00,900
4,5975.00,1500
5,13800.00,2000
6,24800.00,4000
10,940.00,1
11,4225.00,6500
12,7980.00,400


In [15]:
# 1 = HardCore Buyer
# 2 = Middle Range Buyer
# 3 = Prospect

# Segment category / Value association
def ScoreConv2(x):
    if x == '111':
        return int(1)
    elif x == '444':
        return int(3)
    elif x == '443': 
        return int(3)
    elif x == '344': 
        return int(3)
    elif x == '222': 
        return int(2)
    elif x == '211': 
        return int(1)
    elif x == '122': 
        return int(2)
    elif x == '322': 
        return int(2)
    elif x == '244': 
        return int(3)
    elif x == '343': 
        return int(3)   

In [16]:
#-----------------------------------------------------------------------------------------------------------------

rfmTable['Profile_Cat'] = rfmTable['RFMScore'].apply(lambda x: ScoreConv2(x))
rfmTable['Profile_Cat'] = rfmTable['Profile_Cat'].astype(int)

In [17]:
rfmTable.head()

,Recency,Frequency,Monetary_value,Total_Quant,RFMScore,RFM_Cat,Profile_Cat
CustomerID,,,,,,,
12347.0,2,7,4310.00,2458,111,1,1
12350.0,310,1,334.40,197,443,3,3
12352.0,36,7,1665.74,533,211,6,1
12353.0,204,1,89.00,20,444,2,3
12355.0,214,1,459.40,240,443,3,3


In [18]:
# X train
data_client_rfm = rfmTable.filter(items=(['Total_Quant',
                                          'Monetary_value']))

scaler_client = preprocessing.StandardScaler().fit(data_client_rfm)
X_train = scaler_client.transform(data_client_rfm) 

# X for prediction
data_client_rfm_pred = rfmTable_pred.filter(items=(['Total_Quant',
                                                    'Monetary_value']))

scaler_client2 = preprocessing.StandardScaler().fit(data_client_rfm_pred)
X = scaler_client2.transform(data_client_rfm_pred)

In [19]:
# y train

y_train = rfmTable.filter(items=(['Profile_Cat']))
y_train = y_train['Profile_Cat'].ravel()

array([1, 3, 1, ..., 3, 3, 1])

In [20]:
#Random Forest with best parameters
rfc = RandomForestClassifier(n_estimators= 100, max_depth = 3)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
data_client_rfm_pred['Profile_Cat_Pred'] = rfc.predict(X)

In [22]:
# Score RFM / Value association
def ConvProfile(x):
    if x == 1:
        return str('Hardcore Buyer')
    elif x == 2:
        return str('Middle Range Buyer')
    elif x == 3: 
        return str('Prospect')

In [23]:
data_client_rfm_pred['Pred_Profile'] = data_client_rfm_pred['Profile_Cat_Pred'].apply(lambda x: ConvProfile(x))

In [24]:
#-----------------------------------------------------------------------------------------------------------------
# Results Display

data_client_rfm_pred

,Total_Quant,Monetary_value,Profile_Cat_Pred,Pred_Profile
CustomerID,,,,
1,5,15.47,3,Prospect
2,200,955.00,3,Prospect
3,900,3815.00,3,Prospect
4,1500,5975.00,2,Middle Range Buyer
5,2000,13800.00,1,Hardcore Buyer
6,4000,24800.00,1,Hardcore Buyer
10,1,940.00,3,Prospect
11,6500,4225.00,3,Prospect
12,400,7980.00,2,Middle Range Buyer


In [25]:
#-----------------------------------------------------------------------------------------------------------------
# Results Saving

data_client_rfm_pred.to_csv(path_or_buf='DATA_test/data_Segmentation_Results.csv')